# OCR Integration for Scanned Documents

Optical Character Recognition (OCR) allows you to extract text from scanned documents where the text isn't embedded in the PDF. This tutorial demonstrates how to work with scanned documents.

In [1]:
#%pip install "natural-pdf[all]"

In [2]:
from natural_pdf import PDF

# Load a PDF
pdf = PDF("https://github.com/jsoma/natural-pdf/raw/refs/heads/main/pdfs/needs-ocr.pdf")
page = pdf.pages[0]

# Try extracting text without OCR
text_without_ocr = page.extract_text()
f"Without OCR: {len(text_without_ocr)} characters extracted"

'Without OCR: 0 characters extracted'

## Applying OCR and Finding Elements

The core method is `page.apply_ocr()`. This runs the OCR process and adds `TextElement` objects to the page. You can specify the engine and languages.

**Note:** Re-applying OCR to the same page or region will automatically remove any previously generated OCR elements for that area before adding the new ones.

In [3]:
# Apply OCR using the default engine (EasyOCR) for English
page.apply_ocr(languages=['en'])

# Select all text pieces found by OCR
text_elements = page.find_all('text[source=ocr]')
print(f"Found {len(text_elements)} text elements using default OCR")

# Visualize the elements
text_elements.highlight()

# Apply OCR using PaddleOCR for English and Chinese
page.apply_ocr(engine='paddle', languages=['en', 'ch_sim'])

# Apply OCR using SuryaOCR for English and German
page.apply_ocr(engine='surya', languages=['en', 'de'])

text_with_ocr = page.extract_text()
print(f"\nExtracted text after OCR:\n{text_with_ocr[:150]}...")

Rendering pages:   0%|          | 0/1 [00:00<?, ?it/s]

2025-04-25T21:09:07.381722Z [warning  ] Using CPU. Note: This module is much faster with a GPU. lineno=71 module=easyocr.easyocr


[2025-04-25 17:09:07,381] [ WARNING] easyocr.py:71 - Using CPU. Note: This module is much faster with a GPU.


Found 47 text elements using default OCR


Rendering pages:   0%|          | 0/1 [00:00<?, ?it/s]

...

[2025-04-25 17:09:30,624] [   ERROR] ocr_manager.py:236 - An unexpected error occurred during OCR processing: You need to pass in one list of languages for each image
Traceback (most recent call last):
  File "/Users/soma/Development/natural-pdf/natural_pdf/ocr/ocr_manager.py", line 195, in apply_ocr
    results = engine_instance.process_image(
  File "/Users/soma/Development/natural-pdf/natural_pdf/ocr/engine.py", line 117, in process_image
    raw_results = self._process_single_image(processed_img, detect_only, options)
  File "/Users/soma/Development/natural-pdf/natural_pdf/ocr/engine_surya.py", line 71, in _process_single_image
    results = self._recognition_predictor(
  File "/Users/soma/.pyenv/versions/3.10.13/lib/python3.10/site-packages/surya/recognition/__init__.py", line 44, in __call__
    assert len(images) == len(langs), "You need to pass in one list of languages for each image"
AssertionError: You need to pass in one list of languages for each image


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/soma/Development/natural-pdf/natural_pdf/core/pdf.py:390 in apply_ocr                     │
│                                                                                                  │
│    387 │   │   ocr_start_time = time.monotonic()                                                 │
│    388 │   │   try:                                                                              │
│    389 │   │   │   # Manager's apply_ocr signature needs to accept common args directly          │
│ ❱  390 │   │   │   batch_results = self._ocr_manager.apply_ocr(**manager_args)                   │
│    391 │   │   │                                                                                 │
│    392 │   │   │   if not isinstance(batch_results, list) or len(batch_results) != len(images_p  │
│    393 │   │   │   │   logger.error(                                                             │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │  apply_exclusions = True                                                                     │ │
│ │       detect_only = False                                                                    │ │
│ │            device = None                                                                     │ │
│ │                 e = AssertionError('You need to pass in one list of languages for each       │ │
│ │                     image')                                                                  │ │
│ │            engine = 'surya'                                                                  │ │
│ │   failed_page_num = 1                                                                        │ │
│ │  final_resolution = 150                                                                      │ │
│ │                 i = 0                                                                        │ │
│ │        images_pil = [<PIL.Image.Image image mode=RGB size=1275x1651 at 0x372345030>]         │ │
│ │               img = <PIL.Image.Image image mode=RGB size=1275x1651 at 0x372345030>           │ │
│ │         languages = ['en', 'de']                                                             │ │
│ │      manager_args = {                                                                        │ │
│ │                     │   'images': [                                                          │ │
│ │                     │   │   <PIL.Image.Image image mode=RGB size=1275x1651 at 0x372345030>   │ │
│ │                     │   ],                                                                   │ │
│ │                     │   'engine': 'surya',                                                   │ │
│ │                     │   'languages': ['en', 'de'],                                           │ │
│ │                     │   'detect_only': False                                                 │ │
│ │                     }                                                                        │ │
│ │    min_confidence = None                                                                     │ │
│ │     ocr_call_args = {'engine': 'surya', 'languages': ['en', 'de'], 'detect_only': False}     │ │
│ │    ocr_start_time = 227631.441174166                                                         │ │
│ │           options = None                                                                     │ │
│ │              page = <Page number=1 index=0>                                                  │ │
│ │    page_image_map = [                                                                        │ │
│ │                     │   (                                                                    │ │
│ │                     │   │   <Page number=1 index=0>,     

2025-04-25T21:09:30.699050Z [error    ] Batch OCR processing failed: You need to pass in one list of languages for each image lineno=403 module=natural_pdf.core.pdf



[2025-04-25 17:09:30,699] [   ERROR] pdf.py:403 - Batch OCR processing failed: You need to pass in one list of languages for each image
Traceback (most recent call last):
  File "/Users/soma/Development/natural-pdf/natural_pdf/core/pdf.py", line 390, in apply_ocr
    batch_results = self._ocr_manager.apply_ocr(**manager_args)
  File "/Users/soma/Development/natural-pdf/natural_pdf/ocr/ocr_manager.py", line 195, in apply_ocr
    results = engine_instance.process_image(
  File "/Users/soma/Development/natural-pdf/natural_pdf/ocr/engine.py", line 117, in process_image
    raw_results = self._process_single_image(processed_img, detect_only, options)
  File "/Users/soma/Development/natural-pdf/natural_pdf/ocr/engine_surya.py", line 71, in _process_single_image
    results = self._recognition_predictor(
  File "/Users/soma/.pyenv/versions/3.10.13/lib/python3.10/site-packages/surya/recognition/__init__.py", line 44, in __call__
    assert len(images) == len(langs), "You need to pass in one li


Extracted text after OCR:
...


## Advanced OCR Configuration

For more control, import and use the specific `Options` class for your chosen engine within the `apply_ocr` call.

In [4]:
from natural_pdf.ocr import PaddleOCROptions, EasyOCROptions, SuryaOCROptions

# Re-apply OCR using EasyOCR with specific options
easy_opts = EasyOCROptions(
    paragraph=False,
)
page.apply_ocr(engine='easyocr', languages=['en'], min_confidence=0.1, options=easy_opts)

paddle_opts = PaddleOCROptions(
    use_angle_cls=False,
    det_db_thresh=0.3,
)
page.apply_ocr(engine='paddle', languages=['en'], options=paddle_opts)

surya_opts = SuryaOCROptions()
page.apply_ocr(engine='surya', languages=['en'], min_confidence=0.5, detect_only=True, options=surya_opts)

Rendering pages:   0%|          | 0/1 [00:00<?, ?it/s]

Rendering pages:   0%|          | 0/1 [00:00<?, ?it/s]

Rendering pages:   0%|          | 0/1 [00:00<?, ?it/s]

Detecting bboxes:   0%|                                      | 0/1 [00:00<?, ?it/s]

Detecting bboxes: 100%|██████████████████████████████| 1/1 [00:00<00:00,  1.68it/s]

Detecting bboxes: 100%|██████████████████████████████| 1/1 [00:00<00:00,  1.68it/s]

<Page number=1 index=0>

## Interactive OCR Correction / Debugging

If OCR results aren't perfect, you can use the bundled interactive web application (SPA) to review and correct them.

1.  **Package the data:**
    After running `apply_ocr` (or `apply_layout`), use `create_correction_task_package` to create a zip file containing the PDF images and detected elements.

    ```python
    from natural_pdf.utils.packaging import create_correction_task_package

    page.apply_ocr()

    create_correction_task_package(pdf, "correction_package.zip", overwrite=True)
    ```

2.  **Run the SPA:**
    Navigate to the SPA directory within the installed `natural_pdf` library in your terminal and start a simple web server.

3.  **Use the SPA:**
    Open `http://localhost:8000` in your browser. Drag the `correction_package.zip` file onto the page to load the document. You can then click on text elements to correct the OCR results.

## Working with Multiple Pages

Apply OCR or layout analysis to all pages using the `PDF` object.

In [5]:
# Process all pages in the document

# Apply OCR to all pages (example using EasyOCR)
pdf.apply_ocr(engine='easyocr', languages=['en'])
print(f"Applied OCR to {len(pdf.pages)} pages.")

# Or apply layout analysis to all pages (example using Paddle)
# pdf.apply_layout(engine='paddle')
# print(f"Applied Layout Analysis to {len(pdf.pages)} pages.")

# Extract text from all pages (uses OCR results if available)
all_text_content = pdf.extract_text(page_separator="\\n\\n---\\n\\n")

print(f"\nCombined text from all pages:\n{all_text_content[:500]}...")

Rendering pages:   0%|          | 0/1 [00:00<?, ?it/s]

2025-04-25T21:09:59.535433Z [warning  ] Ignoring unsupported layout keyword argument: 'page_separator' lineno=57 module=natural_pdf.utils.text_extraction


[2025-04-25 17:09:59,535] [ WARNING] text_extraction.py:57 - Ignoring unsupported layout keyword argument: 'page_separator'


Applied OCR to 1 pages.

Combined text from all pages:
Red (ZGB tuple]
Jungle Health and Satety Inspection Service
INS-UPONSINCLAIR
Site: Durham's Meatpacking Chicago, IIl.
Date: February 3, 1905
Violation Count:
Summary: Worst of any, however; were the fertilizer men, and those who served in the cooking rooms
These people could not be shown to the visitorfor the odor of a fertilizer man would scare anyordinary
visitor at a hundred yards, and as for the other men, who worked in tank rooms full of steam, and in
some of which there were open vats near...


## Saving PDFs with Searchable Text

After applying OCR to a PDF, you can save a new version of the PDF where the recognized text is embedded as an invisible layer. This makes the text searchable and copyable in standard PDF viewers.

Use the `save_searchable()` method on the `PDF` object:

In [6]:
from natural_pdf import PDF

input_pdf_path = "https://github.com/jsoma/natural-pdf/raw/refs/heads/main/pdfs/needs-ocr.pdf"

pdf = PDF(input_pdf_path)
# Apply OCR to all pages before saving
# Use desired engine and options
pdf.apply_ocr(engine='easyocr', languages=['en'])

pdf.save_searchable("needs-ocr-searchable.pdf")

print("Saved searchable PDF to needs-ocr-searchable.pdf")

Rendering pages:   0%|          | 0/1 [00:00<?, ?it/s]

2025-04-25T21:09:59.784298Z [warning  ] Using CPU. Note: This module is much faster with a GPU. lineno=71 module=easyocr.easyocr


[2025-04-25 17:09:59,784] [ WARNING] easyocr.py:71 - Using CPU. Note: This module is much faster with a GPU.


Saved searchable PDF to needs-ocr-searchable.pdf


This creates `needs-ocr-searchable.pdf`, which looks identical to the original but now has a text layer corresponding to the OCR results. You can adjust the rendering resolution used during saving with the `dpi` parameter (default is 300).

OCR integration enables you to work with scanned documents, historical archives, and image-based PDFs that don't have embedded text. By combining OCR with natural-pdf's layout analysis capabilities, you can turn any document into structured, searchable data. 